In [11]:
import buffer, expr, reader, repl, utils

# PyCombinator Interpreter

We'll build PyCombinator, our own basic Python Interpreter. By the end of the lab, we'll be able to use primitives such as `add`, `mul`, `sub`, and we'll be able to create and call lambda functions.

We'll implement some of the key parts that will allow us to evaluate the following commands and more:

In [2]:
from operator import *

In [3]:
add(3, 4)

7

In [4]:
mul(4, 5)

20

In [5]:
sub(2, 3)

-1

In [6]:
(lambda: 4)()

4

In [7]:
(lambda x, y: add(y, x))(3, 5)

8

In [8]:
(lambda x: lambda y: mul(x, y))(3)(4)

12

In [9]:
(lambda f: f(0))(lambda x: pow(2, x))

1

We can find the **Read-Eval-Print Loop** code for our interpreter in `repl.py`. Here is an overview of eachof the REPL components:


**1.** **Read**: The function `read` in `reader.py` calls the following 2 functions to parse user input.
* The **lexer** is the function **tokenize** in **reader.py** which splits the user input string into tokens
* The **parser** is the function **read_expr** in **reader.py** which parses the tokens and turns expressions into instances of subclasses of the class **Expr** in **expr.py** (e.g. **CallExpr**)

**2.** **Eval**: Expressions (represented as `Expr` objects) are evaluated to obtain values (represented as `Value` objects, also in `expr.py`)
* Eval: Each type of expression has its own **eval** method which is called to evaluate it
* Apply: Call expressions are evaluated by calling the operator's **apply** method on the arguments. For lambda procedures, **apply** calls **eval** to evaluate the body of the function.
* Print: The `__str__` representation of the obtained value is printed.

In this lab, we'll only be implementing the `Eval` and `Apply` steps in `expr.py`. We can start the PyCombinator interpreter by running the following command:

In [ ]:
# Assume this is a terminal
$ python3 repl.py

Right now, any names (e.g. `add`) and call expressions (e.g. `add(2, 3)`) will output `None`. It's our job to implement `Name.eval` and `CallExpr.eval` so that we can look up names and call functions in our interpreter!

In [ ]:
>>> add
None
>>> add(2, 3)
None

We don't have to understand how the `read` component of our interpreter is implemented, but if we want a better idea of how user input is read and transformed into Python code, we can use the `--read` flag when running the interpreter.

In [ ]:
# Assume this is a terminal
$ python3 repl.py --read
>>> add
Name('add')
>>> 3
Literal(3)
>>> lambda x: mul(x, x)
LambdaExpr(['x'], CallExpr(Name('mul'), [Name('x'), Name('x')]))
>>> add(2, 3)
CallExpr(Name('add'), [Literal(2), Literal(3)])

## Q4: Prologue
Here is the breakdown of our implementations:

**1.** `repl.py` contains the logic for the REPL loop, which repeatedly reads expressions as user input, evaluates them, and prints out their values (we don't have to completely understand all the code in this file)

**2.** `reader.py` contains our interpreter's reader. The function `read` calls the function `tokenize` and `read_expr` to turn an expression string into an `Expr` object (we don't have to completely understand all the code in this file).

**3.** `expr.py` contains our interpreter's representation of expressions and values. The subclasses of `Expr` and `Value` encapsulate all the types of expressions and values in the PyCombinator language. The global environment, a dictionary containing the bindings for primitive functions, is also defined at the bottom of this file.



In [ ]:
$ python3 ok -q prologue_reader -u

Q: What does REPL stand for?

0. Really-Enormous-Purple-Llamas
1. Read-Eval-Parse-Lex
2. Read-Eval-Print-Loop

Ans: 2

Q: What does the `read` component of the REPL loop do?

0. Evaluates call expressions
1. Turns input into a useful data structure
2. Turns input into tokens
3. Ensures a function has been defined before it is called

Ans: 1

What does the tokenize function in `reader.py` return?

0. Input expression represented as a list of tokens
1. Input expression represented as an instance of a subclass of `Expr`
2. Input expression with corrected number of parentheses
3. Result of evaluating the input expression

Ans: 0

What will tokenize('add(3, 4)') output?

0. `['add', '(', 3, ',', 4, ')']`
1. `['a', 'd', 'd', '(', 3, ',', 4, ')']`
2. `['add', '(', '3', ',', '4', ')']`
3. `['a', 'd', 'd', '(', '3', ',', '4', ')']`

Ans: 0

What will `tokenize('(lambda: 4)()')` output?

0. `['(', LambdaExpr, 4, ')', '(', ')']`
1. `['(', LambdaExpr, ':', 4, ')', '(', ')']`
2. `['lambda', 4, '(', ')']`
3. `['(', 'lambda', ':', 4, ')', '(', ')']`

Ans: 3

What does the `read_expr` function in `reader.py` accept as input and return?  (looking at the read function may help answer this question)

0. List of tokens and an instance of a subclass of `Expr`
1. Input expression and list of tokens
2. List of tokens and number of parentheses
3. Input expression and an instance of a subclass of `Expr`

Ans: 0

What does the read function in reader.py return?

0. Input expression with corrected number of parentheses
1. Result of evaluating the input expression
2. Input expression represented as an instance of a subclass of Expr
3. Input expression represented as a list of tokens

Ans: 2

What will read('1') output?

0. `Name('1')`
1. `Name(1)`
2. `Literal(1)`
3. `Number(1)`

Ans: 2

What will read('x') output?


0. `Name(x)`
1. `Name('x')`
2. `Literal(x)`
3. `x`

What will read('add(3, 4)') output?


0. `CallExpr(Literal('add'), Literal(3), Literal(4))`
1. `CallExpr(Name('add'), Literal(3), Literal(4))`
2. `CallExpr(Name('add'), [Literal(3), Literal(4)])`
3. `CallExpr('add', [Literal(3), Literal(4)])`

Ans: 2

In [ ]:
$ python ok -q prologue_expr -u

What are all the types of expressions in PyCombinator?

0. name, function, number, literal
1. value, expression, function, number
2. number, lambda function, primitive function, string
3. literal, name, call expression, lambda expression

Ans: 3

What are all the types of values in PyCombinator?

0. number, lambda expression, primitive function
1. name, number, lambda function
2. number, lambda function, primitive function
3. number, string, function

Ans: 2

What does a Literal evaluate to?

0. a Number
1. a String
2. an Expression
3. a Function

Ans: 0

Q: What is the difference between a lambda expression and a lambda function?

0. They are the same thing
1. A lambda function is the result of evaluating a lambda expression
2. A lambda expression is a call to a lambda function
3. A lambda expression is the result of evaluating a lambda function

Ans: 1

Which of the following describes the eval method?

0. A method of LambdaExpression objects that evaluates a function call
1. A method of Expr objects that evaluates the Expr and returns a Value
2. A method of Expr objects that evaluates a call expression and returns a Number
3. A method of Literal objects that returns a Name

Ans: 1

How are environments represented in our interpreter?

0. As sequences of Frame objects
1. As dictionaries that map variable names (strings) to Value objects
2. As dictionaries that map Name objects to Value objects
3. As linked lists containing dictionaries

Ans: 1